In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
tf.random.set_seed(13)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install tensorflow==2.9.2

In [ ]:
!pip install keras

In [ ]:
print(tf.__version__)

In [ ]:
data_desempleo = pd.read_csv('desempleo.csv', sep=';')
print(data_desempleo.head())

In [ ]:
TargetVariable=['unemployment']
Predictors = ['minimum_salary', 'gdp', 'gfcf']
X=data_desempleo[Predictors].values
y=data_desempleo[TargetVariable].values

from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
 
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)
 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1120, shuffle=False)


In [ ]:
def make_regression_ann(units_trial1, units_trial2, Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    model = Sequential()
    model.add(Dense(units=units_trial1, input_dim=3, activation='relu'))
    model.add(Dense(units=units_trial2, input_dim=3, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

Parameter_Trials={'batch_size':[2, 4, 8, 10, 12, 14],
                    'epochs':[32, 34, 36, 60, 65, 70, 75, 80],
                    'units_trial1':[2, 3, 4, 5, 6, 7, 8],
                    'units_trial2':[2, 3, 4, 5, 6, 7, 8],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }

In [ ]:
RegModel=KerasRegressor(make_regression_ann, verbose=0)

In [ ]:
from sklearn.metrics import make_scorer
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

In [ ]:
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

In [ ]:
import time
StartTime=time.time()
grid_search.fit(X,y, verbose=1)
EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')
print('### Printing Best parameters ###')
grid_search.best_params_

In [ ]:
#train the model with the best parameters
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('desempleo.csv', sep=';')
print(df.head())

In [ ]:
y = df['unemployment']
X = df.drop(['date', 'inflation', 'unemployment'], axis=1)
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.10,
                                                    random_state = 1120, shuffle=False)

min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [ ]:
#GridSearchCV {'Optimizer_trial': 'rmsprop', 'batch_size': 4, 'epochs': 65, 'units_trial1': 8, 'units_trial2': 7}
model = Sequential()
model.add(Dense(8, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='linear')) 
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    epochs=65,
                    batch_size=4,
                    verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and validation loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2)

axes[0].scatter(x=y_train, y=model.predict(X_train))
axes[0].set_xlabel("Actual", fontsize=10)
axes[0].set_ylabel("Predicted",  fontsize=10)
axes[0].set_title("Training")

x = np.linspace(*axes[0].get_xlim())
axes[0].plot(x, x, color='red')

axes[1].scatter(x=y_test, y=model.predict(X_test))
axes[1].set_xlabel("Actual", fontsize=10)
axes[1].set_ylabel("Predicted",  fontsize=10)
axes[1].set_title("Validation")

x = np.linspace(*axes[1].get_xlim())
axes[1].plot(x, x, color='red')

fig.tight_layout()
plt.show()

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
#ANN (2L)
y = np.array(y_test)  #predictor variable
yhat = yhat = np.array(pred) #predicted variable

#MSE
m_mse=mean_squared_error(y,yhat)
#MAE
mae_error = mae(y, yhat)
#MAPE
def MAPE(y,yhat):
    mape = np.mean(np.abs((y - yhat)/y))*100
    return mape
print('Test MSE: ', m_mse)
print('Test MAE: ', mae_error)
print('Test MAPE: ', MAPE(y,yhat))